In [34]:
# import pandas as pd
# log_type = 'test'
# df = pd.read_csv(f"experiment_log_{log_type}.csv")

# df.tail(2)

In [35]:
# df.to_csv(f'./experiment_log_{log_type}.csv',index=False)

In [36]:
# import kaggle
# from kaggle.api.kaggle_api_extended import KaggleApi

# # Authenticate
# api = KaggleApi()
# api.authenticate()

In [37]:
# import pandas as pd
# from datetime import datetime
# import pytz

# # 1. Define the Singapore Timezone
# sg_tz = pytz.timezone('Asia/Singapore')

# # 2. Your date string in 'yyyy-mm-dd hh:mm:ss' format
# # Note: I corrected 'hh:ss:hh' to 'hh:mm:ss' (hours:minutes:seconds)
# date_string = '2026-01-18 00:00:00'

# # 3. Parse the string into a naive datetime object
# naive_date = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S')

# # 4. Localize it to Singapore Time, then convert to UTC
# # This ensures the comparison with Kaggle (UTC) is accurate
# cutoff_date_sg = sg_tz.localize(naive_date)
# cutoff_date_utc = cutoff_date_sg.astimezone(pytz.UTC)
# cutoff_date_naive = cutoff_date_utc.replace(tzinfo=None)
# print(cutoff_date_naive)

# # The competition ID/slug from the URL (e.g., 'titanic')
# comp_name = 'amex-default-prediction'

# # Retrieve submissions
# submissions = api.competition_submissions(comp_name)

# # Convert the list of submission objects into a Pandas DataFrame
# # Each object has attributes like 'ref', 'date', 'description', 'status', 'publicScore', 'privateScore'
# data = []
# for sub in submissions:
#     if sub.date > cutoff_date_naive:
#         data.append({
#             'date': sub.date,
#             'description': sub.description,
#             'status': sub.status,
#             'public_score': sub.public_score,
#             'private_score': sub.private_score
#         })

# df = pd.DataFrame(data)

# # Save to CSV
# df.to_csv('my_kaggle_submissions.csv', index=False)
# df.head(2)

In [38]:
# import json
# import pandas as pd

# # 1. Convert the description string into a Python dictionary/list
# # We use a lambda to handle potential nulls or malformed strings safely
# def parse_json(x):
#     try:
#         data = json.loads(x)
#         # If the JSON is wrapped in a list [{}], take the first element
#         return data[0] if isinstance(data, list) else data
#     except:
#         return {}

# # 2. Apply the parsing logic
# json_struct = df['description'].apply(parse_json)

# # 3. Create a new DataFrame from the JSON objects and join it back
# df_expanded = pd.json_normalize(json_struct)

# # 4. Combine with your original stats
# # We drop the original 'description' to keep it clean
# final_df = pd.concat([df.drop(columns=['description']), df_expanded], axis=1)

# final_df.head()

In [39]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticate
api = KaggleApi()
api.authenticate()

In [40]:
import pandas as pd
import json
from datetime import datetime
import pytz

def download_sub(date_string = '2026-01-18 06:00:00' ):
    # --- 1. Setup Cutoff (Example for Jan 17, 2026 SGT) ---
    sg_tz = pytz.timezone('Asia/Singapore')
    naive_date = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S')
    cutoff_date_utc = sg_tz.localize(naive_date).astimezone(pytz.UTC).replace(tzinfo=None)

    # --- 2. Fetch and Process ---
    submissions = api.competition_submissions('amex-default-prediction')
    processed_rows = []

    for sub in submissions:
        if sub.date > cutoff_date_utc:
            try:
                # Parse the JSON from the description
                desc_raw = json.loads(sub.description)
                # Handle list-wrapped JSON: [{"key": "val"}]
                desc_dict = desc_raw[0] if isinstance(desc_raw, list) else desc_raw
                
                # --- 3. RECONSTRUCT ROW IN JSON ORDER ---
                row = {}
                for key in desc_dict.keys():
                    if key == 'public_score':
                        # Use the official API score in the JSON's 'public_score' slot
                        row[key] = sub.public_score
                    elif key == 'private_score':
                        # Use the official API score in the JSON's 'private_score' slot
                        row[key] = sub.private_score
                    else:
                        # Keep the original JSON value for all other keys
                        row[key] = desc_dict[key]
                
                # Safety check: if the JSON didn't actually contain the score keys,
                # you can optionally append them here:
                if 'public_score' not in row: row['public_score'] = sub.public_score
                if 'private_score' not in row: row['private_score'] = sub.private_score

                processed_rows.append(row)
                
            except Exception as e:
                # Skip rows where description is not valid JSON
                continue

    # --- 4. Create DataFrame ---
    # Since Python 3.7+, dictionaries preserve insertion order, 
    # and Pandas will honor the order of the keys from the first row.
    final_df = pd.DataFrame(processed_rows)
    return final_df

In [41]:
final_df = download_sub(date_string = '2026-01-19 05:00:00' )
final_df = final_df.dropna(subset=['feature_w'])
final_df.to_csv(f'./experiment_log_submission.csv',index=False)
final_df.tail(2)

,model_specs,log_time,feature_w,fcst_w,recon_w,att_w,lr,sampling,data_type,seed,...,emb_version,remark,is_predict,amex_metric,AUC,test_start_time,test_end_time,test_duration,public_score,private_score
46,original_10pct_0.0002_42_128_3_512_1_0.2_1e-06...,2026-01-21 15:44:36,0.000001,0.000001,1.0,1.0,0.0002,10pct,original,42,...,NaN,NaN,True,None,None,2026-01-21 15:36:11,2026-01-21 15:44:36,505165,0.70317,0.71164
47,original_10pct_0.002_42_128_3_512_1_0.2_1e-06_...,2026-01-21 15:22:55,0.000001,1.000000,1.0,0.0,0.0020,10pct,original,42,...,NaN,NaN,True,None,None,2026-01-21 15:14:35,2026-01-21 15:22:55,500897,0.71805,0.72889


In [42]:
# 0. Filter Specific Rows
final_df2 = final_df[final_df['remark'] == 'only forecast loss'].copy()

# 1. Ensure numeric type
final_df2['private_score'] = pd.to_numeric(final_df2['private_score'], errors='coerce')
final_df2.tail(2)

,model_specs,log_time,feature_w,fcst_w,recon_w,att_w,lr,sampling,data_type,seed,...,emb_version,remark,is_predict,amex_metric,AUC,test_start_time,test_end_time,test_duration,public_score,private_score
29,original_1pct_0.0001_42_128_3_512_1_0.2_0.01_1...,2026-01-30 06:46:44,0.01,1.0,0.5,0.01,0.0001,1pct,original,42,...,v2,only forecast loss,True,None,None,2026-01-30 06:38:20,2026-01-30 06:46:44,504368,0.62454,0.63677
30,original_1pct_0.001_42_128_3_512_1_0.2_0.01_1....,2026-01-30 06:27:07,0.01,1.0,0.5,0.01,0.0010,1pct,original,42,...,v2,only forecast loss,True,None,None,2026-01-30 06:18:40,2026-01-30 06:27:07,506953,0.68638,0.69335


In [43]:
# 1. Sort by score (descending)
# 2. Group by your breakdown
# 3. Take top n of each group
cols = [    'feature_w', 'fcst_w', 'recon_w', 'att_w',
            # 'lr', 
            'sampling']

(final_df2.sort_values('sampling', ascending=False)
         .groupby(cols)
         .head(2)
         [cols + ['private_score']] # Select just the columns you want to see
)

,feature_w,fcst_w,recon_w,att_w,sampling,private_score
30,0.01,1.0,0.5,0.01,1pct,0.69335
29,0.01,1.0,0.5,0.01,1pct,0.63677
23,0.01,1.0,0.5,0.01,10pct,0.73549
28,0.01,1.0,0.5,0.01,10pct,0.73946
14,0.01,1.0,0.5,0.01,100pct,0.75845
19,0.01,1.0,0.5,0.01,100pct,0.74256


In [46]:
# 0. Filter Specific Rows
final_df2 = final_df[final_df['remark'].str.contains('Amodel_CAI', na=False)].copy()

# 1. Ensure numeric type
final_df2['private_score'] = pd.to_numeric(final_df2['private_score'], errors='coerce')
final_df2.tail(2)

,model_specs,log_time,feature_w,fcst_w,recon_w,att_w,lr,sampling,data_type,seed,...,emb_version,remark,is_predict,amex_metric,AUC,test_start_time,test_end_time,test_duration,public_score,private_score
11,original_10pct_0.001_42_128_3_512_1_0.2_0.01_1...,2026-02-08 17:28:25,0.01,1.0,0.5,0.01,0.001,10pct,original,42,...,v2,"only forecast loss, test Amodel_CAI",True,None,None,2026-02-08 17:21:09,2026-02-08 17:28:25,436078,0.77368,0.78338
12,original_1pct_0.001_42_128_3_512_1_0.2_0.01_1....,2026-02-08 17:17:21,0.01,1.0,0.5,0.01,0.001,1pct,original,42,...,v2,"only forecast loss, test Amodel_CAI",True,None,None,2026-02-08 17:10:04,2026-02-08 17:17:21,436849,0.75119,0.76156


In [51]:
# 1. Sort by score (descending)
# 2. Group by your breakdown
# 3. Take top n of each group
cols = [    'feature_w', 'fcst_w', 'recon_w', 'att_w',
            # 'lr', 
            'sampling']

(final_df2.sort_values('sampling', ascending=False)
         .groupby(cols)
         .head(2)
         [cols + ['private_score']] # Select just the columns you want to see
)


,feature_w,fcst_w,recon_w,att_w,sampling,private_score
5,0.01,1.0,0.5,0.01,1pct,0.73369
6,0.01,1.0,0.5,0.01,1pct,0.76188
1,0.01,1.0,0.5,0.01,10pct,0.78192
2,0.01,1.0,0.5,0.01,10pct,0.78513
0,0.01,1.0,0.5,0.01,100pct,0.79877
